# Decoradores

¿Quién podría explicar lo que está pasando aquí?:

### Funciones (first-class object)

In [4]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(funcion):
    return funcion("Bob")

In [5]:
greet_bob(say_hello)

'Hello Bob'

In [3]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

In [6]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

In [13]:
def parent(num):
    def first_child(n):
        return "Hi, I am Emma", n

    def second_child(n):
        return "Call me Liam", n

    if num == 1:
        return first_child
    else:
        return second_child

In [14]:
first = parent(1)
second = parent(2)


# descomentar las siguientes líneas y mirad que va pasando
# first
# second

# first()
# second()

In [20]:
second(2)

('Call me Liam', 2)

### Un decorador básico

In [21]:
def my_decorator_printantesdespues(func):
    def wrapper():
        print("Ocurre algo ANTES de llamar a la función.")
        print(f"el valor de retorno es: {func()}")
        print("Ocurre algo DESPUÉS de llamar a la función.")

    return wrapper

In [22]:
def say_whee():
    print("Whee!")

In [23]:
say_whee()

Whee!


In [24]:
say_whee_decorada = my_decorator_printantesdespues(say_whee)

In [25]:
say_whee_decorada()

Ocurre algo ANTES de llamar a la función.
Whee!
el valor de retorno es: None
Ocurre algo DESPUÉS de llamar a la función.


In [26]:
@my_decorator_printantesdespues
def say_whee():
    print("Whee!")

In [27]:
@my_decorator_printantesdespues
def funcion():
    
    return "cadena"

In [31]:
funcion()

Ocurre algo ANTES de llamar a la función.
el valor de retorno es: cadena
Ocurre algo DESPUÉS de llamar a la función.


In [32]:
@my_decorator_printantesdespues
def decir_hola():
    print("Holaaa")

In [33]:
decir_hola()

Ocurre algo ANTES de llamar a la función.
Holaaa
el valor de retorno es: None
Ocurre algo DESPUÉS de llamar a la función.


In [45]:
permitido = True


def pedir_permiso(func):
    def wrapper():
        if permitido:
            print("Adelante!")
            return func()
        else:
            print("Pues va a ser que no...")
            pass  # No se puede :(

    return wrapper


@pedir_permiso
def hacer_algo():
    return "Voy a bañarme!"

In [46]:
permitido = True


resultado = hacer_algo()

Ocurre algo ANTES de llamar a la función.
Adelante!
el valor de retorno es: Voy a bañarme!
Ocurre algo DESPUÉS de llamar a la función.


In [44]:
resultado

'Voy a bañarme!'

In [77]:
# decorado que permite argumentos
def pedir_permiso_arg(permitido=True):
    def decorador_real(func):
        def wrapper(*args, **kwargs):
            if permitido:
                print("Adelante!")
                func(*args, **kwargs)
            else:
                print("Pues va a ser que no...")
                pass  # No se puede :(

        return wrapper
    
    return decorador_real


@pedir_permiso_arg()
def hacer_algo():
    print("Voy a bañarme!")

In [78]:
hacer_algo()

Adelante!
Voy a bañarme!


In [64]:
def f(*args, **kwargs):
    print(args)
    print(kwargs)

In [68]:
f(1, 2, 3, nombre="ricardo")

(1, 2, 3)
{'nombre': 'ricardo'}


In [56]:
d = {"a": 1, "v": 2}

In [59]:
def f2(a, v):
    print(a)
    print(v)

In [60]:
f2(**d)

1
2


## \*args y \*\*kwargs

In [46]:
def f(*args, **kwargs):
    print(args)
    print(kwargs)

In [48]:
f(1,2,3,4, ultimo=5)

(1, 2, 3, 4)
{'ultimo': 5}


Como hacer correctamente un decorador que acepte argumentos pero que funcione también si no le pasamos ningun argumento. Es decir, los argumentos son opcionales.

In [ ]:
# source: https://florimond.dev/blog/articles/2019/09/python-optionally-parametrized-decorators/
# also Python Cookbook: Recipes for Mastering Python 3
# Book by Brian K. Jones and David M. Beazley
# https://stackoverflow.com/questions/48098569/use-of-functools-partial-in-a-decorator-that-attaches-function-as-attribute-of-o

import functools


def logged(func=None, decimals: int = None):
    if func is None:
        print(f"Called with decimals={decimals}")
        return functools.partial(logged, decimals=decimals)

    print(f"Called without parameters, func={func}.")

    @functools.wraps(func)
    def decorated(*args, **kwargs):
        print(f"{func.__name__} called with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        logged_result = result if decimals is None else round(result, decimals)
        print(f"Result: {logged_result}")
        return result

    return decorated

In [42]:
from datetime import datetime
import functools

def not_during_the_night(func):
    @functools.wraps(func)
    def wrapper():
        if 20 <= datetime.now().hour < 11:
            func()
        else:
            print("es de noche")
            pass
    return wrapper


def say_whee():
    """Dice whee."""
    
    print("Whee!")

# no permite ejecutar la función a ciertas horas
say_whee = not_during_the_night(say_whee)

In [39]:
say_whee()

es de noche


In [20]:
say_whee()

es de noche


`*args` y `**kwargs`

## Ejercicio 01

Crear un decorador llamado `@timer`.

El decorados me dice el tiempo que ha tardado en ejecutarse está funcion (usando `print()`).

In [ ]:
@timer
def hola():
    print("jnkjn")

In [ ]:
hola()

### ¿Qué es eso de @functools.wrap?

`@functools.wraps` es un decorador que preserva la información de la función original (metadatos: docstring, etc)

### Usando clases para crear decoradores

La mayoría de las veces no será necesario 🙅🏻‍♀️

¿Cómo haríais un decorador que almacena el número de veces que la función ha sido llamada?

In [21]:
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def haz_algo():
    print("Whee!")

In [22]:
import time

In [23]:
haz_algo()

Call 1 of 'haz_algo'
Whee!


El método `.__ init __ ()` debe almacenar una referencia a la función y puede hacer cualquier otra inicialización necesaria. Se llamará al método `.__ call __ ()` en lugar de la función decorada. Hace lo mismo que la función `wrapper()` en nuestros ejemplos anteriores. Ten en cuenta que se usa la función `functools.update_wrapper()` en lugar de `@functools.wraps`.

In [24]:
# source: https://python-3-patterns-idioms-test.readthedocs.io/en/latest/PythonDecorators.html#what-can-you-do-with-decorators
# PythonDecorators/decorator_with_arguments.py

class decorador_con_argumentos(object):
    def __init__(self, arg1, arg2, arg3):
        """
        If there are decorator arguments, the function
        to be decorated is not passed to the constructor!
        """
        print("Inside __init__()")
        self.arg1 = arg1
        self.arg2 = arg2
        self.arg3 = arg3

    def __call__(self, f):
        """
        If there are decorator arguments, __call__() is only called
        once, as part of the decoration process! You can only give
        it a single argument, which is the function object.
        """
        print("Inside __call__()")

        def wrapped_f(*args):
            print("Inside wrapped_f()")
            print("Decorator arguments:", self.arg1, self.arg2, self.arg3)
            f(*args)
            print("After f(*args)")

        return wrapped_f


@decorador_con_argumentos("hello", "world", 42)
def sayHello(a1, a2, a3, a4):
    print("sayHello arguments:", a1, a2, a3, a4)


print("After decoration")

print("Preparing to call sayHello()")
sayHello("say", "hello", "argument", "list")
print("after first sayHello() call")
sayHello("a", "different", "set of", "arguments")
print("after second sayHello() call")

Inside __init__()
Inside __call__()
After decoration
Preparing to call sayHello()
Inside wrapped_f()
Decorator arguments: hello world 42
sayHello arguments: say hello argument list
After f(*args)
after first sayHello() call
Inside wrapped_f()
Decorator arguments: hello world 42
sayHello arguments: a different set of arguments
After f(*args)
after second sayHello() call


Decorador básico:

In [118]:
def decor(func):
    def wrapper(*args, **kwargs):
        
        result = func(*args, **kwargs)

        return result
        
    return wrapper

In [119]:
@decor
def suma(a, b):
    return a + b

In [120]:
res = suma(1,2)
res

(1, 2)


3

### Ejercicios 🎉

2. `@do_twice`: ejecutar la función 2 veces  
2. `@timer`: contar cuanto tiempo tarda en ejecutarse la función  
2. `@logger`: guardar nombre de la función y hora en una lista  

3. `@slowdown`: parar un tiempo X entre cada ejecución  
4. `@cache`: guardar el valor de la función para esos argumentos  
  * Hacer que esa la cache de la función se guarde en un archivo `.json`
  * ¿Alguien puede conseguir lo mismo con solo 1 línea de código?

5. `@repeat(n=4)`: igual que `@do_twice` pero ahora podemos elegir el número de veces que queremos que se repita

*Extra:*  
 * `@debug`: hacer print de los argumentos con los que se ha llamado la función y el nombre de la función cada vez que se llama

Más información y fuentes:

* https://realpython.com/primer-on-python-decorators 👈🏼
* https://dbader.org/blog/python-decorators
* https://blog.miguelgrinberg.com/post/the-ultimate-guide-to-python-decorators-part-i-function-registration
* https://stackoverflow.com/questions/10294014/python-decorator-best-practice-using-a-class-vs-a-function 📖
* https://www.youtube.com/watch?v=MjHpMCIvwsY 📹

Ricardo Ander-Egg Aguilar

* 🖥: https://ricardoanderegg.com/
* 🐦: https://twitter.com/ricardoanderegg
* 👨🏻‍🎓: https://www.linkedin.com/in/ricardoanderegg/